In [1]:
from eunjeon import Mecab
import re

In [2]:
#coherency
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
def split_to_sentences(text):
    """
    Args: 
        text (str): raw text to split to sentences on end of sentences marks.
    Returns:
        List of sentences from text.
    """
    # Split on end of sentence, but keep the punctuation marks.
    text = text.replace('\n', '')
    sentences = text.split('.')
    # If the last sentence is ''
    if len(sentences) > 1 and len(sentences[-1]) < 3:
        sentences.pop()
    return sentences

def split_words(text):
    return re.split(r'[ ](?=[\w])', text)

def remove_stopwords(text):
    stopwords = ['에게', '가', '은', '는', '이', '을', '를']
    text = split_words(text)
    filtered_word_sets = [word for word in text if not word in stopwords]
    return filtered_word_sets

In [6]:
ex_text = '''
옛날 옛적에 한 오누이가 살고 있었습니다. 한 어머니는 빨갛고 예쁜 옷에 낡았으면 팔려고 했어요. 하지만 손에 꼭 맞는 옷을 만들지 못했해서 결국 엄마는 빨간 헝겊 저고리를 쓰게 되었죠. 어느날 오빠는 새어머니를 모시고 숲으로 갔답니다. 도깨비 할아버지가 방바닥에 앉아 이삭을 하고 있는데 까마귀들이 입을 크게 벌리고 쏟아져 나릅니다 어머나 사람들은 깜짝 놀랐지 뭐예요? 그때 여우가 물었어요. 왜 그래! 옛날 하늘나라 임금님이 눈치채도록 까만 손수레를 밀고 가버렸기에여...임금님은 그 광경을 가만히 지켜보고 있었던 거야. 아니, 저렇게 납작하게 입어봐라 거짓말처럼 큰소리로 말했거든요. 그 모습을 본 임금은 얼굴을 찌푸리며 그만 울상이 되고 말아졌어, 얼굴에는 웃음이 암까지 나왔지만 간신히 몸을 다 펴보았을 때 머리 위에 하얀 구름이 덩쿨이 서고, 앞은 흰 구름 속으로 사라져 버렸다고 하죠. 오빠와 엄마 이야기는 이렇게 이야기를 들으면서 그동안 궁전으로 돌아온 이야기와 앞으로 나갈 일을 이야기 하였답니다.
'''

In [47]:
ex_text_list = [
    '옛날 옛적, 어느 마을에 아주 사이좋은 형제가 살고 있었어요. 형은 결혼하여 가족들과 살고 있었고 동생은 어머니를 모시고 있었지요. 두 형제는 매일 들에 나가 열심히 일을 했어요. 가을이 되자, 논에는 누런 벼가 금빛으로 출렁거렸어요. 형제는 즐거운 마음으로 추수를 하였지요. 그리고는 쌀을 나누었어요. 쌀가마가 창고에 가득하게 쌓였어요. 이것을 본 형은 마음이 몹씨 흐뭇했어요. 그런데 문득 이런 생각이 들었어요.‘날씨가 좋은 날은 형님이 볏단을 가져다 놓고 쉬고 가시는 날인가 봐.’ 곡간이 열린 날, 형은 볏단을 지고 아우네 집으로 갔어요.형이 볏단을 지고 아우는 얼굴을 내밀자마자 아우는 기가 막힌 생각이 들었어요. ‘아우야, 이게 어떻게 된 일이야?’ 하고 아우가 속으로 생각했어요. ‘형님은 내가 잘못 한 거 같구나.’ 동생은 집으로 돌아와 가족들에게 자신이 지은 볏단을 보여주었어요.그러다 우연히도 형과 아우는 볏단이 어떻게 되었는지 알게 되었다.서로에게 이야기를 나누던 중 형과 아우는 서로에게 좋은 일이 생길 것 같다며 밤새도록 이야기를 나눌 수 있게 되었어요. 형제는 다시 서로에게 달려들며 위험에서 구해주기 위해 달려갔어요.그제야 형제는 자신들의 실수를 깨달았어요.',
    '옛날옛적, 어느 마을에 아주 사이좋은 형제가 살고 있었어요. 형은 결혼하여 가족들과 살고 있었고 동생은 어머니를 모시고 있었지요. 두 형제는 매일 들에 나가 열심히 일을 했어요. 가을이 되자, 논에는 누런 벼가 금빛으로 출렁거렸어요. 형제는 즐거운 마음으로 추수를 하였지요. 그리고는 쌀을 나누었어요. 쌀가마가 창고에 가득하게 쌓였어요. 이것을 본 형은 마음이 몹씨 흐뭇했어요. 그런데 문득 이런 생각이 들었어요.‘날씨가 좋은 날은 형님이 볏단을 가져다 놓고 쉬고 가시는 날인가 봐.’ 곡간이 열린 날, 형은 볏단을 지고 아우네 집으로 갔어요.형이 볏단을 지고 아우는 얼굴을 내밀자마자 아우는 기가 막힌 생각이 들었어요. ‘아우야, 이게 어떻게 된 일인가?’ 하고 아우가 속으로 생각했어요.마침 눈을 떠보니 자신이 볏단을 지고 개울가에 무릎을 탁탁 깔고 앉았지요. ‘오늘은 마침 발이 개미가 조용히 지나가는 소리가 들렸어요.',
    ex_text
]

# Coherency
- 문단의 맥락을 평가한다.
- LSA를 이용해 첫번째 문장과 마지막 문장의 유사도를 측정한다.

In [15]:
def _coherency(text, embedder):
    texts_sentences = split_to_sentences(text)
    transformed_sentences = embedder.fit_transform(texts_sentences)
    similarity = cosine_similarity(transformed_sentences)
    return sum(similarity[0][1:])

In [16]:
embedder = TfidfVectorizer()

In [19]:
_coherency(ex_text, embedder)

0.11463666872271021

# Readability
- 읽기 쉬운 글인지 평가한다.
- 점수가 높을수록 독자 연령대가 높은 텍스트이다.

In [20]:
def _readabilty(text):
    """
    Uses length of sentences and length of words.
    Higher is for more advanced readers.
    If text is sparse i.e. mostly new lines, and doesn't end with an eos -> add a negative cost.  
    Args:
        text (str): original text to return score for.
        texts_sentences (list): text split to sentences. 
    """
    texts_sentences = split_to_sentences(text)
    txt_words = split_words(text)
    num_letters = sum(len(word) for word in txt_words)
    num_words = len(txt_words)
    num_sent = len(texts_sentences)

    # check if a "sparse" sentence
    if num_sent == 1:
        new_line_threshold = 0 if num_words == 0 else num_words // 4
        if texts_sentences[0].count('\n') > new_line_threshold or not re.search(r'(?<![A-Z])[.!?;"]+', texts_sentences[0]):
            num_sent = 0

    letters_per_word = -10 if num_words == 0 else num_letters/num_words
    words_per_sentence = -10 if num_sent == 0 else num_words/num_sent
    # 0.5 to weight words_per_sentence higher
    return 0.5*letters_per_word + words_per_sentence

In [21]:
 _readabilty(ex_text)

11.972666666666665

# Diversity
- stopwords에 들어간 표현을 제외하고, 문장에서 얼마나 다양한 표현을 써냈는지 평가한다.
- 점수가 높을수록 다양성이 높은 것이다.

In [22]:
def remove_stopwords(text):
    text = text.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    tokenizer = Mecab('C:\mecab\mecabrc').morphs
    tokens = tokenizer(text)
    stopwords = ['은','는','이','가']
    
    text = [word for word in tokens if word not in stopwords]
    return text

In [23]:
def _diversity(text):
    """
    Fraction of unique words from the total number of words (exclusing stop words).
    Higher is more diversified.
    Args:
        filtered_words (list): set of non-stop tokenized words. 
        filtered_words_set (set): unique filtered words.
    """
    filtered_words = remove_stopwords(text)
    filtered_words_set = set(filtered_words)
    
    # If empty sentence or only white space or \n or too repetitive.
    if len(filtered_words_set) < 5:
        return 0

    return len(filtered_words_set) / len(filtered_words)

In [24]:
_diversity(ex_text)

0.6608695652173913

# Simplicity
- fine tuned 모델에서 named entity를 제외한 most freq words 정의 필요.
- 동화에서 가장 자주 사용되는 unique words set과의 교집합을 구한다.

In [25]:
SEVEN_PREC_MOST_FREQ_WORDS = [
    '꿀꿀', '쿵쿵', '와르르'
]

In [26]:
def _simplicity(text):
    """
    Fraction of most frequent words from generated text.
    Args:
        filtered_words_set (set): set of non-stop, non-punctuation words. 
    """
    filtered_words = remove_stopwords(text)
    filtered_words_set = set(filtered_words)
    
    return len(filtered_words_set.intersection(SEVEN_PREC_MOST_FREQ_WORDS))

In [27]:
_simplicity(ex_text)

0

# Bias

# Ranking
- 위 평가 지표들을 통해 생성된 문장들의 순위를 매겨 1순위를 최종 output으로 쓴다.

In [37]:
import numpy as np

In [74]:
def score_text(text_list):
    score = []
    for i in range(len(text_list)):
        text = text_list[i]
        text_sentences = split_to_sentences(text)
        each_score=[]
        each_score.append(_coherency(text, embedder))
        each_score.append(_readabilty(text))
        each_score.append(_diversity(text))
        each_score.append(_simplicity(text))
        score.append(each_score)
    stories_scores = np.array(score)

    
    stories_scores_normalized = stories_scores - np.min(stories_scores, axis=0)
    min_max_denominator = np.max(stories_scores, axis=0) - np.min(stories_scores, axis=0)
   
    stories_scores_normalized = np.divide(
        stories_scores_normalized, min_max_denominator, out=np.zeros_like(stories_scores_normalized), where=min_max_denominator != 0)
    rank = np.argsort(np.mean(stories_scores_normalized, axis=1))[::-1]
    return rank

In [75]:
score_text(ex_text_list)

array([2, 1, 0], dtype=int64)